In [1]:
import json
from rich.pretty import pprint
from phi.agent import Agent
from phi.model.ollama import OllamaTools

In [2]:
OLLAMA_HOST="localhost"
#OLLAMA_HOST="ollama.ollama.svc.cluster.local"

First we'll create our own tool for structured extraction like in the previous notebook

In [3]:
from phi.tools import Toolkit
from phi.model.ollama import Ollama
from phi.utils.log import logger

class NuExtractTools(Toolkit):
    def __init__(self, json_template: str):
        super().__init__(name="nuextract_tools")
        self.json_template = json_template
        self.extract_agent = Agent(
            model=Ollama(id="sroecker/nuextract-tiny-v1.5", host=OLLAMA_HOST, options={"temperature": 0}),
            markdown=False,
            structured_outputs=True
        )
        self.register(self.predict_nuextract)

        
    def predict_nuextract(self, input_text: str) -> str:
        """Extracts structured information and returns JSON.
        
        Args:
            input_text (str): The input text to extract information from.
        Returns:
            str: The output in JSON.
        """
        template = f"""<|input|>\n### Template:\n{self.json_template}\n### Text:\n{input_text}\n\n<|output|>"""
        logger.info(f"Extracting structured data according to schema: {self.json_template}")
        output_json = self.extract_agent.run(template, stream=False)
        
        return output_json.content

In [4]:
json_template = """
    {
        "Model": {
            "Name": "",
            "Number of parameters": "",
            "Number of max token": "",
            "Architecture": []
        },
        "Usage": {
            "Use case": [],
            "Licence": ""
        }
    }
    """

In [5]:
example_text = """We introduce Mistral 7B, a 7–billion-parameter language model engineered for
superior performance and efficiency. Mistral 7B outperforms the best open 13B
model (Llama 2) across all evaluated benchmarks, and the best released 34B
model (Llama 1) in reasoning, mathematics, and code generation. Our model
leverages grouped-query attention (GQA) for faster inference, coupled with sliding
window attention (SWA) to effectively handle sequences of arbitrary length with a
reduced inference cost. We also provide a model fine-tuned to follow instructions,
Mistral 7B – Instruct, that surpasses Llama 2 13B – chat model both on human and
automated benchmarks. Our models are released under the Apache 2.0 license.
Code: <https://github.com/mistralai/mistral-src>
Webpage: <https://mistral.ai/news/announcing-mistral-7b/>"""

Now we're using a normal LLM that can use the NuExtract tool that we just implemented:

In [6]:
extract_agent = Agent(
    model=Ollama(id="hermes3:8b-llama3.1-q8_0", host=OLLAMA_HOST),
    description="You extract structured information.",
    tools=[NuExtractTools(json_template)],
    instructions=["Extract and output structured information from the provided text with your tools."],
    show_tool_calls=True,
    debug_mode=True,
    markdown=True,
    structured_outputs=False
)

DEBUG    Debug logs enabled

In [7]:
pprint(extract_agent.print_response(example_text))

DEBUG    *********** Agent Run Start: 6ffdbc1c-c9d2-4f9a-aa18-6caf9a8f99ea ***********

DEBUG    Function predict_nuextract from nuextract_tools added to model.

DEBUG    ---------- Ollama Response Start ----------

DEBUG    ============== system ==============

DEBUG    You extract structured information.                                                                       
                                                                                                                   
         ## Instructions                                                                                           
         - Extract and output structured information from the provided text with your tools.                       
         - Use markdown to format your answers.

...

DEBUG    Added 4 Messages to AgentMemory

DEBUG    Added AgentRun to AgentMemory

DEBUG    --**-- Logging Agent Run

DEBUG    *********** Agent Run End: 6ffdbc1c-c9d2-4f9a-aa18-6caf9a8f99ea ***********

None